In [3]:
import praw
import openai
import os
import pandas as pd

# Replace the following values with your own credentials
REDDIT_CLIENT_ID = "INSERT"
REDDIT_SECRET = "INSERT"
REDDIT_USER_AGENT = "INSERT "
REDDIT_USERNAME = "INSERT"
REDDIT_PASSWORD = "INSERT"

OPENAI_API_KEY = "INSERT"

# Initialize an empty DataFrame to store the results
df = pd.DataFrame(columns=["post_id", "title", "eval_chatgpt", "result_community"])

# Initialize Reddit API with PRAW
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT,
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
)

# Initialize OpenAI API
openai.api_key = OPENAI_API_KEY

# Function to evaluate if OP is an asshole using ChatGPT
def is_op_an_asshole(post_text):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": """You are a moral philosopher.
         Evaluate the following post on r/amitheasshole.  
        Determine whether the OP (original poster) of this post is the asshole in the scenario they describe.
        Provide your reasoning in two paragraphs. End your response with a categorical outcome:
        
        "YTA": OP is the asshole in the scenario.
        "NTA": OP is not the asshole in the scenario.
        "No A-holes here": There are no assholes in the scenario.
        "Everyone Sucks": Everyone's an asshole in the scenario.
         """},
        {"role": "user", "content": f"{post_text}"}
    ],
    temperature=.4)
    answer = response['choices'][0]['message']['content']
    return answer

# Retrieve all posts from r/amitheasshole
subreddit = reddit.subreddit("amitheasshole")

# Replace 'hot' with 'new', 'rising', or 'controversial' to change the sorting of posts
for post in subreddit.top(limit=5):
    post_title = post.title
    post_selftext = post.selftext
    post_text = f"Title: {post_title}\n\n{post_selftext}"
    post_flair = post.link_flair_text.lower()

    # Check if OP is an asshole according to ChatGPT
    if post_flair not in ['update', 'meta']:
        eval_chatgpt = is_op_an_asshole(post_text)
    else:
        eval_chatgpt = 'N/A'
    # Check if OP is an asshole according to community flair
    if post_flair:
        result_community = post_flair.lower()
    else:
        result_community = "Unknown"
    # Add the results to the DataFrame
    post_results_df = pd.DataFrame({
        "post_id": [post.id],
        "title": [post_title],
        "eval_chatgpt": [eval_chatgpt],
        "result_community": [result_community]
    })

    # Append the new DataFrame to the main DataFrame using pandas.concat
    df = pd.concat([df, post_results_df], ignore_index=True)


In [1]:
import pmaw

In [4]:
df

,post_id,title,eval_chatgpt,result_community
0,ocx94s,AITA for telling my wife the lock on my daughter's door does not get removed til my brother inlaw and his daughters are out of our house?,"The OP is not the asshole in this scenario. It is understandable that they would want to protect their daughter's privacy and belongings, especially after the incident with the expensive makeup kit. The fact that the wife and brother-in-law do not see an issue with the behavior of their daughters is concerning and shows a lack of respect for Zoey's boundaries.\n\nThe wife's reaction to the lock and her insistence that the OP treat Sammy's daughters like his own is unreasonable. It is not fair to expect the OP to prioritize the comfort of Sammy's family over the well-being of his own daughter. Additionally, the wife's attempt to shift blame onto her own family for not taking in Sammy and his daughters is irrelevant to the current situation and does not excuse her behavior.\n\nOverall, the OP is justified in wanting to protect his daughter's privacy and belongings and is not the asshole in this scenario. The wife and brother-in-law need to acknowledge and respect Zoey's boundaries and work towards a solution that works for everyone.",not the a-hole
1,d6xoro,META: This sub is moving towards a value system that frequently doesn't align with the rest of the world,N/A,meta
2,azvko1,"UPDATE, AITA for despising my mentally handicap sister?",N/A,update
3,gr8bp3,AITA For suing my girlfriend after she had my 1967 impala project taken to the scrapyard?,"In this scenario, the OP is not the asshole. The girlfriend's actions were not only disrespectful but also illegal. The OP had invested a significant amount of time, effort, and money into restoring the car, and the girlfriend had no right to dispose of it without the OP's consent. Moreover, the girlfriend's behavior was manipulative, as she waited until the OP was out of town to carry out her plan.\n\nThe fact that the car was an old piece of junk is irrelevant. The value of the car to the OP was not just monetary but also sentimental, as evidenced by the time and effort invested in the restoration process. The girlfriend's family and friends are wrong to dismiss the OP's feelings and to suggest that he should let it go.\n\nIn conclusion, the girlfriend is the asshole in this scenario, and the OP is justified in taking legal action against her.",not the a-hole
4,x2k5kv,AITA for bringing my SIL’s wallet to the restaurant when she conveniently always forgets it?,"OP is the asshole in this scenario. While it is understandable that they are frustrated with their SIL's behavior of conveniently forgetting her wallet and expecting OP to pay for expensive meals, taking her wallet without her knowledge or consent is a violation of her privacy and trust. It is not a solution to the problem and only exacerbates the tension between them.\n\nFurthermore, the fact that OP admits to getting the idea from a TV show suggests a lack of consideration for the seriousness of the situation. This behavior sets a bad example for others and undermines the trust and respect that should be present in familial relationships.\n\nA better approach would be to have an honest conversation with the SIL about the financial strain that her behavior is causing and come to a mutually agreeable solution. Taking matters into one's own hands in such a deceitful manner is not a mature or ethical way to handle the situation. Therefore, OP is the asshole in this scenario.",not the a-hole
